In [13]:
pip install tqdm


In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.layers import Input, Dense, LeakyReLU
from keras.models import Model
from keras.optimizers import Adam
from tqdm import tqdm

# Define the path to the CSV file
data_file_path = "C:/Users/wdd45/OneDrive/바탕 화면/딥러닝응용/WISDM.csv"

# Load the data from the CSV file
data = pd.read_csv(data_file_path)


In [15]:
# Balance data for 'Sitting' and 'Jogging'
sitting_data = data[data['activity'] == 'Sitting'].head(4500).copy()
jogging_data = data[data['activity'] == 'Jogging'].head(4500).copy()

In [16]:
# Concatenate the balanced data
balanced_data = pd.concat([sitting_data, jogging_data])


In [17]:
# Encode the 'activity' column using LabelEncoder
label_encoder = LabelEncoder()
balanced_data['activity'] = label_encoder.fit_transform(balanced_data['activity'])


In [18]:
# Select only the specified columns
selected_columns = ['activity', 'X', 'Y', 'Z']
balanced_data = balanced_data[selected_columns]


In [19]:
# Display the resulting DataFrame
print(balanced_data)


        activity         X          Y         Z
221333         1  2.600000   9.660000  1.035146
221334         1  2.680000   9.530000  0.503953
221335         1  3.490000   8.890000  0.762740
221336         1  3.640000   9.380000  0.926184
221337         1  2.910000   9.340000  1.035146
...          ...       ...        ...       ...
13015          0  5.434519   8.853226  3.909040
13016          0 -4.331271  19.531578 -1.729784
13017          0 -1.035146  -5.012288 -4.481094
13018          0  7.082581  17.243360 -0.381370
13019          0  6.442424   5.053149 -5.788648

[9000 rows x 4 columns]


In [20]:
# Split the data into training and testing sets
X_train, X_test = train_test_split(balanced_data, test_size=0.25, random_state=42)


In [21]:
# Separate 'Normal' data for training
normal = X_train[X_train['activity'] == 0]
y_train = normal['activity']
X_train_normal_train = normal.drop(['activity'], axis=1)


In [22]:
# Separate target and features for testing
y_test = X_test['activity']
X_test = X_test.drop(['activity'], axis=1).values


In [23]:
# GAN Model

In [24]:
# Generator
generator_input_dim = 100
generator_input = Input(shape=(generator_input_dim,))
generator_output = Dense(32, activation='relu')(generator_input)
generator_output = Dense(3, activation='tanh')(generator_output)  # 3 output dimensions (X, Y, Z)
generator = Model(generator_input, generator_output)


In [25]:
# Discriminator
discriminator_input_dim = X_train_normal_train.shape[1]
discriminator_input = Input(shape=(discriminator_input_dim,))
discriminator_output = Dense(32)(discriminator_input)
discriminator_output = LeakyReLU(alpha=0.2)(discriminator_output)
discriminator_output = Dense(1, activation='sigmoid')(discriminator_output)
discriminator = Model(discriminator_input, discriminator_output)
discriminator.compile(optimizer=Adam(lr=0.0002, beta_1=0.5), loss='binary_crossentropy', metrics=['accuracy'])

C:\Users\wdd45\anaconda3\envs\py37\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [26]:
# Combined GAN Model
discriminator.trainable = False
gan_input = Input(shape=(generator_input_dim,))
x = generator(gan_input)
gan_output = discriminator(x)
gan = Model(gan_input, gan_output)
gan.compile(optimizer=Adam(lr=0.0002, beta_1=0.5), loss='binary_crossentropy')

In [27]:
# Training the GAN
epochs = 10
batch_size = 32

In [28]:
batch_count = X_train_normal_train.shape[0] // batch_size


In [29]:
for epoch in range(epochs):
    for _ in tqdm(range(batch_count)):
        noise = np.random.normal(0, 1, size=[batch_size, generator_input_dim])
        generated_data = generator.predict(noise)
        real_data = X_train_normal_train.iloc[np.random.randint(0, X_train_normal_train.shape[0], size=batch_size)]

        X = np.concatenate([real_data, generated_data])
        y_dis = np.zeros(2 * batch_size)
        y_dis[:batch_size] = 0.9  # One-sided label smoothing

        discriminator.trainable = True
        d_loss = discriminator.train_on_batch(X, y_dis)

        noise = np.random.normal(0, 1, size=[batch_size, generator_input_dim])
        y_gen = np.ones(batch_size)

        discriminator.trainable = False
        g_loss = gan.train_on_batch(noise, y_gen)

    print(f"Epoch {epoch}, Discriminator Loss: {d_loss[0]}, Generator Loss: {g_loss}")


  0%|                                                                                          | 0/105 [00:00<?, ?it/s]

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
1/1 [==============================] - 0s 141ms/step
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report 

  1%|▊                                                                                 | 1/105 [00:00<01:34,  1.10it/s]

1/1 [==============================] - 0s 27ms/step


  3%|██▎                                                                               | 3/105 [00:01<00:31,  3.22it/s]

...

1/1 [==============================] - 0s 22ms/step


 99%|███████████████████████████████████████████████████████████████████████████████▏| 104/105 [00:11<00:00, 12.81it/s]

1/1 [==============================] - 0s 21ms/step


100%|████████████████████████████████████████████████████████████████████████████████| 105/105 [00:11<00:00,  9.33it/s]

Epoch 9, Discriminator Loss: 0.39261412620544434, Generator Loss: 1.2431950569152832


In [30]:
# Evaluate on the test set
generated_data_test = generator.predict(np.random.normal(0, 1, size=[X_test.shape[0], generator_input_dim]))

71/71 [==============================] - 0s 1ms/step


In [31]:
# Reconstruction error (MSE) between generated data and real data
reconstruction_error = np.mean(np.square(X_test - generated_data_test))

In [32]:

print(f"Reconstruction Error on Test Set: {reconstruction_error}")


Reconstruction Error on Test Set: 46.13330431139045
